## Hyperparameters

In [1]:
P = {
    "pre_Scaler" : "std",           # (None, std, minmax)
    "pre_QuantiletTrans" : True, 
    "data_BatchSize" : 125,
    "data_NbFolds" : 10,
    "data_NbValidFolds" : 2,
    "data_UpSampleCoeff" : 0.10,    # add label 1 data x coeff
    "data_DownSampleCoeff" : 0.95,  # multiply label 0 by coeff
    "mod_Act" : "leaky",            # (relu, leaky, tanh, sig) tanh/no
    "mod_ActSlope" : 0.02,          # for leaky
    "mod_NbLayers" : 3, 
    "mod_BatchNorm" : "after",      # empty, before or after = after for relu ?!
    "mod_LayerCoeff": 0.9, 
    "mod_Dropout": 0.45, 
    "mod_HiddenSize": 520,          # 520
    "mod_InputSize": 200,           # 200+ for data augmentation
    "train_MaxLoop":   7,           # 
    "train_MaxEpoch": 100,          # max epoch avant stop (1 pour test / 80 déjà atteint avec reg L1)
    "train_Loss": "bce",            # bce or mse
    "train_CountDown": 5,
    "train_AugLossAlpha": 0.02,     #  
    "train_LbdL1": 0.00001,         # 0.0001, 0.00005 0.0001 peut être OK, voir à la baisse 
    "train_LbdL2": 0.00005,          # converge : mse=0.0032, 0.0001 impact faible 
    "train_LbdAUC": 0.00010,         # 0.1000 stupid idea.... (should be with a big batch size for AUC calculation)
    "train_Opt": "SGD",             # SGD or Adam
    "train_LearnRate": 0.001,
    "train_SGDmomentum": 0.92,      # for SGD : room for value between 0.9 and <1.0
    "valid_Metric": "auc",       # "L1loss" oo "auc"
    "train_BoostingValid":  50,     # Augment validation data where failed
    "train_BoostingTrain":  50,     # Augment training data where failed
}
# note try boosting = upsmapling top error / balanced class / per fold...

P_Boosting_Top_valid = P["train_BoostingValid"]
P_Boosting_Top_train = P["train_BoostingTrain"]
P_Boosting = False
if P_Boosting_Top_valid > 0 or P_Boosting_Top_train > 0:
    P_Boosting = True
# P_Index = 43


In [2]:
import pandas as pd
import numpy as np
import random, pickle 


from sklearn.naive_bayes import GaussianNB
import lightgbm as lgb
import xgboost as xgb

import torch
from torch import nn, optim
import torch.nn.functional as F

from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
from sklearn.metrics import roc_auc_score



In [3]:
# if P_Boosting == False:
df_grid = pd.read_csv("grid.csv", index_col=0)
df_grid = df_grid.append(pd.Series(P), ignore_index=True)
df_grid.to_csv("grid.csv", index=True)
grid_index = max(df_grid.index)
P_Index = grid_index
# else:
#     grid_index = P_Index
# print(grid_index)

In [4]:
df_train = pd.read_csv('train.csv.zip', float_precision="float64")
df_test = pd.read_csv('test.csv.zip', float_precision="float64")
df = pd.concat([df_train, df_test], ignore_index=True, sort=True)

df_X = df.drop(columns=["ID_code", "target"])
Y = df_train["target"].values.astype('float64')
print("done.")

done.


In [5]:
if P["pre_Scaler"] == "std":
    scaler = StandardScaler()
    df_Xn = pd.DataFrame(scaler.fit_transform(df_X))
if P["pre_Scaler"] == "minmax":
    scaler = MinMaxScaler()
    df_Xn = pd.DataFrame(scaler.fit_transform(df_X))
if P["pre_Scaler"] == "none":
    df_Xn = df_X.copy()
print("done.")

done.


In [6]:
if P["pre_QuantiletTrans"] : 
    df_Xt = pd.DataFrame(QuantileTransformer(output_distribution='normal').fit_transform(df_Xn))
else:
    df_Xt = df_Xn.copy()
print("done.")

done.


In [7]:
idx = []
idx = [i for i in range(200_000)]
random.shuffle(idx)
X_train = df_Xt.values.astype('float64')[:200_000] 
Y_train = df_train["target"].values.astype('float64')
X_train = [X_train[i] for i in idx] 
Y_train = [Y_train[i] for i in idx]
X_train = np.array(X_train) 
Y_train = np.array(Y_train)
Z_train = df_Xt.values.astype('float64')

gnb = GaussianNB()
gnb.fit(X_train,Y_train)
Y_gnb = gnb.predict_proba(Z_train)[:,1]
df_Xt["gnb"] = Y_gnb
gnb = None
print("done.")

done.


In [8]:
# https://www.kaggle.com/vishalbajaj2000
LGBGBDT_PARAM = {
    'random_state' : 1981,
    'n_estimators' : 2000,
    'learning_rate': 0.1,
    'num_leaves': 16,
    'max_depth': 4,
    'metric' : ['mae'],
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'reg_alpha' : 2.03,
    'reg_lambda' : 4.7,
    'feature_fraction' : 0.8, #colsample_bytree
    'feature_fraction_seed' : 1981, 
    'max_bins' : 100,
    'min_split_gain': 0.0148,
    'min_child_weight' : 7.835, #min_sum_hessian_in_leaf 
    'min_data_in_leaf' : 1000, #min_child_samples
    'random_state' : 1981, # Updated from 'seed'
    'subsample' : .912, #also known as Bagging fraction!
    'subsample_freq' : 200, # also known as bagging frequency!
    'boost_from_average' : False,
    'verbose_eval' : 50,
    'is_unbalance' : True,
    #'scale_pos_weight' : 10.1,
    }

idx = []
idx = [i for i in range(200_000)]
random.shuffle(idx)
X_train = [X_train[i] for i in idx] 
Y_train = [Y_train[i] for i in idx]
X_train = np.array(X_train) 
Y_train = np.array(Y_train)

LGBGBDT = lgb.LGBMClassifier( **LGBGBDT_PARAM, n_jobs=2, verbosity=0)
LGBGBDT_FIT = LGBGBDT.fit(X_train[20_000:200_000],Y_train[20_000:200_000], 
                          eval_set=[(X_train[:20_000],Y_train[:20_000])], 
                          eval_metric= ['auc','binary_logloss'], early_stopping_rounds=100, verbose=False)
Y_lgb = LGBGBDT.predict_proba(Z_train)[:,1]
df_Xt["lgb"] = Y_lgb

LGBGBDT = None
print("done.")

done.


In [9]:
# https://www.kaggle.com/vishalbajaj2000
XGBOOST_PARAM = {
    'random_state' : 1981,
    'n_estimators' : 1000, #very slow with 2000!
    'learning_rate': 0.15,
    'num_leaves': 36,
    'max_depth': 6,
    'metric' : ['auc'],
    'boosting_type' : 'gbdt',
    #'drop_rate' : 0.2,    ##only for DART
    #'max_drop' : 100,    ##only for DART
    #'objective' : 'binary',
    'reg_alpha' : 2.03,
    'reg_lambda' : 4.7,
    'feature_fraction' : 0.8, #colsample_bytree
    'feature_fraction_seed' : 1981, 
    'max_bins' : 100,
    'min_split_gain': 0.0148,
    'min_child_weight' : 7.835, #min_sum_hessian_in_leaf 
    'min_data_in_leaf' : 1000, #min_child_samples
    'random_state' : 1981, # Updated from 'seed'
    'subsample' : .912, #also known as Bagging fraction!
    'subsample_freq' : 200, # also known as bagging frequency!
    'boost_from_average' : False,
    #'verbose_eval' : 5,
    'is_unbalance' : True,
    #'scale_pos_weight' : 10,
    }

idx = []
idx = [i for i in range(200_000)]
random.shuffle(idx)
X_train = [X_train[i] for i in idx] 
Y_train = [Y_train[i] for i in idx]
X_train = np.array(X_train) 
Y_train = np.array(Y_train)

XGBGBDT = xgb.XGBClassifier(**XGBOOST_PARAM, tree_method = 'hist', n_jobs =2, verbosity=0)

XGBGBDT_FIT = XGBGBDT.fit(X_train[20_000:200_000],Y_train[20_000:200_000], 
                          eval_set=[(X_train[:20_000],Y_train[:20_000])],
                          eval_metric='auc', early_stopping_rounds=100, verbose=False)
Y_xgb = XGBGBDT.predict_proba(Z_train)[:,1]
df_Xt["xgb"] = Y_xgb

XGBGBDT = None

X_train = None
Y_train = None
Z_train = None

print("done.")

[08:49:32] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
done.


In [10]:
X = df_Xt.values.astype('float64')[:200_000]
Y = df_train["target"].values.astype('float64')
Z = df_Xt.values.astype('float64')[200_000:]

In [11]:
batch_size = P["data_BatchSize"]
nb_batch_valid = int((200000*0.1)//batch_size)

# idx = []
# filename = "Boosting_idx_{:03}.pickle".format(grid_index)
# if P_Boosting == False:
#     idx = [i for i in range(200_000)]
#     random.shuffle(idx)
#     with open(filename, 'wb') as handle: pickle.dump(idx, handle)
# with open(filename, 'rb') as handle: idx = pickle.load(handle)

idx = []
idx = [i for i in range(200_000)]
random.shuffle(idx)
    
X = [X[i] for i in idx] 
Y = [Y[i] for i in idx]
    
XL = []
YL = []
nb_fold = P["data_NbFolds"]
fold_size = int(len(X)/nb_fold)
for i in range(nb_fold):
    XL.append(X[int(i*fold_size):int((i+1)*fold_size)])
    YL.append(Y[int(i*fold_size):int((i+1)*fold_size)])

nb_fold_valid = P["data_NbValidFolds"]
up_sampling_coeff = P["data_UpSampleCoeff"]
total_1 = [sum(YL[i]) for i in range(nb_fold)]
total_1.sort()
total_1 = total_1[::-1][nb_fold_valid:]
up_sampling_max_size = int(sum(total_1)/len(total_1))
nb_batch_upsampling = int(up_sampling_max_size//batch_size * up_sampling_coeff*(nb_fold-nb_fold_valid))

indice_to_boost = []
for _ in range(nb_fold):indice_to_boost.append([])
# filename = "Boosting_boost_{:03}.pickle".format(grid_index)
# if P_Boosting:
#     with open(filename, 'rb') as handle: indice_to_boost = pickle.load(handle)
# else:
# print(len(indice_to_boost[0]))
# print(idx[0:10]) #[145126, 87119, 48487, 167225, 14305, 113152, 22347, 187537, 13589, 98002]

In [12]:
class Santander(nn.Module):
    def __init__(self):
        super().__init__()
        
        coeff = P["mod_LayerCoeff"]
        nlayers = P["mod_NbLayers"]
        init_size = P["mod_HiddenSize"]
        self.nlayers = nlayers
        self.datasize = P["mod_InputSize"]

        bias=False
        if P["mod_BatchNorm"] == "": 
            bias=True
        
        self.fc1 = nn.Linear(P["mod_InputSize"], int(init_size*(coeff**1)), bias=bias)
        self.bn1 = nn.BatchNorm1d(int(init_size*(coeff**1)))
        out_size = int(init_size*(coeff**1))

        if nlayers > 1:
            in_size = int(init_size*(coeff**1))
            out_size = int(init_size*(coeff**2))
            self.fc2 = nn.Linear(in_size, out_size, bias=bias)
            self.bn2 = nn.BatchNorm1d(out_size)

        if nlayers > 2:
            in_size = int(init_size*(coeff**2))
            out_size = int(init_size*(coeff**3))
            self.fc3 = nn.Linear(in_size, out_size, bias=bias)
            self.bn3 = nn.BatchNorm1d(out_size)
    
        if nlayers > 3:
            in_size = int(init_size*(coeff**3))
            out_size = int(init_size*(coeff**4))
            self.fc4 = nn.Linear(in_size, out_size, bias=bias)
            self.bn4 = nn.BatchNorm1d(out_size)

        if nlayers > 4:
            in_size = int(init_size*(coeff**4))
            out_size = int(init_size*(coeff**5))
            self.fc5 = nn.Linear(in_size, out_size, bias=bias)
            self.bn5 = nn.BatchNorm1d(out_size)

        if nlayers > 5:
            in_size = int(init_size*(coeff**5))
            out_size = int(init_size*(coeff**6))
            self.fc6 = nn.Linear(in_size, out_size, bias=bias)
            self.bn6 = nn.BatchNorm1d(out_size)

        if nlayers > 6:
            in_size = int(init_size*(coeff**6))
            out_size = int(init_size*(coeff**7))
            self.fc7 = nn.Linear(in_size, out_size, bias=bias)
            self.bn7 = nn.BatchNorm1d(out_size)
            
        self.fo = nn.Linear(out_size, 1)

        
        self.activation = nn.ReLU()
        if P["mod_Act"] == "leaky":
            self.activation = nn.LeakyReLU(negative_slope=P["mod_ActSlope"])
        if P["mod_Act"] == "tanh":
            self.activation = nn.Tanh()
        if P["mod_Act"] == "sig":
            self.activation = nn.Sigmoid()
        self.dropout = nn.Dropout(p=P["mod_Dropout"])
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        nlayers = self.nlayers
        x = x.narrow(1,0,self.datasize)
        
        x = self.fc1(x)
        if P["mod_BatchNorm"] == "before":
            x = self.bn1(x)            
        x = self.activation(x)
        if P["mod_BatchNorm"] == "after":
            x = self.bn1(x)            
        x = self.dropout(x)
        
        if nlayers > 1:
            x = self.fc2(x)
            if P["mod_BatchNorm"] == "before":
                x = self.bn2(x)            
            x = self.activation(x)
            if P["mod_BatchNorm"] == "after":
                x = self.bn2(x)            
            x = self.dropout(x)

        if nlayers > 2:
            x = self.fc3(x)
            if P["mod_BatchNorm"] == "before":
                x = self.bn3(x)            
            x = self.activation(x)
            if P["mod_BatchNorm"] == "after":
                x = self.bn3(x)            
            x = self.dropout(x)

        if nlayers > 3:
            x = self.fc4(x)
            if P["mod_BatchNorm"] == "before":
                x = self.bn4(x)            
            x = self.activation(x)
            if P["mod_BatchNorm"] == "after":
                x = self.bn4(x)            
            x = self.dropout(x)
        
        if nlayers > 4:
            x = self.fc5(x)
            if P["mod_BatchNorm"] == "before":
                x = self.bn5(x)            
            x = self.activation(x)
            if P["mod_BatchNorm"] == "after":
                x = self.bn5(x)            
            x = self.dropout(x)

        if nlayers > 5:
            x = self.fc6(x)
            if P["mod_BatchNorm"] == "before":
                x = self.bn6(x)            
            x = self.activation(x)
            if P["mod_BatchNorm"] == "after":
                x = self.bn6(x)            
            x = self.dropout(x)
            
        if nlayers > 6:
            x = self.fc7(x)
            if P["mod_BatchNorm"] == "before":
                x = self.bn7(x)            
            x = self.activation(x)
            if P["mod_BatchNorm"] == "after":
                x = self.bn7(x)            
            x = self.dropout(x)
            
        x = self.fo(x)
        x = self.sigmoid(x)
        
        return x

In [13]:
models_kept = []
for i in range(nb_fold):
    models_kept.append(Santander().double())

criterion = nn.BCELoss()
if P["train_Loss"] == "bce":
    criterion = nn.BCELoss(reduction="none")
#     criterion = nn.BCELoss()
if P["train_Loss"] == "mse":
    criterion = nn.MSELoss(reduction="none")
#     criterion = nn.MSELoss()


In [14]:
def predict(models, X):
    Y_hat = None
    for idx in range(len(X)//batch_size+1):
        data = X[idx*batch_size:(idx+1)*batch_size]
        if len(data) == 0: break
        tData = torch.DoubleTensor(data).cuda()
        Y_hat_batch = None
        for model_idx, model in enumerate(models):
            model = models[model_idx]
            model.eval()
            model.cuda()
            with torch.no_grad():
                predict = model(tData)
                predict = predict.cpu()
            if model_idx == 0 : Y_hat_batch = predict.data
            else: Y_hat_batch += predict.data
        Y_hat_batch = Y_hat_batch/len(models)
        if idx == 0 : Y_hat = Y_hat_batch.numpy()
        else: Y_hat = np.vstack((Y_hat, Y_hat_batch.numpy()))
    return Y_hat

In [15]:

n_epochs = P["train_MaxEpoch"]
countdown_init = P["train_CountDown"]
valid_loss_global_value = [1_000_000.0 for _ in range(nb_fold)]
valid_metric_global_value = [1_000_000.0 for _ in range(nb_fold)]
Lambda_L1 = P["train_LbdL1"]
Lambda_L2 = P["train_LbdL2"]
Lambda_AUC = P["train_LbdAUC"]
recaculate_roc = True
roc = 0.0
for model_loop in range(P["train_MaxLoop"]):
    print("-"*12+str(model_loop+1))
    found_a_better_model = False
    roc_value = 0.0
    countdown = []
    valid_loss_global = 10000.0
    roc_valid_value = 0.0
    valid_loss_local_value = [1_000_000.0 for _ in range(nb_fold)]
    valid_metric_local_value = [1_000_000.0 for _ in range(nb_fold)]
    
    models = []
    for i in range(nb_fold):
        models.append(Santander().double())
    optimizers = []
    for model in models : 
        if P["train_Opt"] == "SGD":
            optimizers.append(torch.optim.SGD(model.parameters(), lr=P["train_LearnRate"], 
                                              momentum=P["train_SGDmomentum"], nesterov=True, 
                                              weight_decay=Lambda_L2))
        if P["train_Opt"] == "Adam":
            optimizers.append(torch.optim.Adam(model.parameters(), lr=P["train_LearnRate"], 
                                              weight_decay=Lambda_L2))
    
    for model in models :
        countdown.append(countdown_init)

    for epoch in range(1, n_epochs+1):

        for model_idx, model in enumerate(models):
            if countdown[model_idx] > 0:

                X_train, Y_train = [], []
                X_valid, Y_valid = [], []
                valid_selected = []
                for j in range(P["data_NbValidFolds"]):
                    valid_selected.append((model_idx+j)%len(models))
                    X_valid += XL[(model_idx+j)%len(models)]
                    Y_valid += YL[(model_idx+j)%len(models)]
                train_selected = [j for j in range(len(models)) if j not in(valid_selected)]
                for j in train_selected:
                    X_train += XL[(j)%len(models)]
                    Y_train += YL[(j)%len(models)]
                    if P_Boosting == True:
                        X_train += [XL[(j)%len(models)][i] for i in indice_to_boost[(j)%len(models)]]
                        Y_train += [YL[(j)%len(models)][i] for i in indice_to_boost[(j)%len(models)]]

                idx_0 = [i for i,y in enumerate(Y_train) if y == 0]
                idx_1 = [i for i,y in enumerate(Y_train) if y == 1]

                #downsampling
                random.shuffle(idx_0)
                idx_0 = idx_0[:int(len(idx_0)*P["data_DownSampleCoeff"])]
                #idx_0 = idx_0[:(len(idx_0)//batch_size)*batch_size]
                X_train = [X_train[i] for i in idx_0] + [X_train[i] for i in idx_1]
                Y_train = [Y_train[i] for i in idx_0] + [Y_train[i] for i in idx_1]

                #upsampling
                idx_1 = [i for i,y in enumerate(Y_train) if y == 1]
                random.shuffle(idx_1)
                idx_temp = []
                for i in range(int(up_sampling_coeff+.999)):
                    idx_temp += idx_1
                idx_1 = idx_temp
                idx_temp = None
                X_up = [X_train[i] for i in idx_1[:nb_batch_upsampling*batch_size]]
                Y_up = [Y_train[i] for i in idx_1[:nb_batch_upsampling*batch_size]]
                X_train += X_up
                Y_train += Y_up

                #shuffling
                idx_loc = [i for i in range(len(X_train))]
                random.shuffle(idx_loc)
                X_train = [X_train[i] for i in idx_loc] 
                Y_train = [Y_train[i] for i in idx_loc]            

                optimizer = optimizers[model_idx]
                train_loss = 0.0
                valid_loss = 0.0
                nb_batch_train = (200_000-fold_size*P["data_NbValidFolds"])//batch_size+nb_batch_upsampling 
                #-nb_batch_valid #+nb_batch_1*1
                nb_batch_train = len(X_train)//batch_size
                model.cuda()

                for idx in range(nb_batch_train):
                    model.train()
                    data = X_train[idx*batch_size:(idx+1)*batch_size]
                    target = Y_train[idx*batch_size:(idx+1)*batch_size]
                    tData = torch.DoubleTensor(data)
                    tTarget = torch.DoubleTensor(target)
                    tData, tTarget = tData.cuda(), tTarget.cuda() 
                    optimizer.zero_grad()
                    outputs = model(tData)
                    loss = criterion(outputs, tTarget.reshape(-1,1))
                    loss = loss + loss * tTarget.reshape(-1,1) * P["train_AugLossAlpha"]
#                     loss = loss.sum() / (1+P["train_AugLossAlpha"])
                    loss = loss*(1+P["train_AugLossAlpha"])

                    if Lambda_AUC > 0 :
                        roc_train = roc_auc_score(tTarget.reshape(-1,1).clone().cpu().data, outputs.clone().cpu().data)
                        AUC_loss = Lambda_AUC*((1-roc_train)/roc_train)
                        loss += loss.clone()*AUC_loss
                    L1_loss = 0
                    if Lambda_L1 > 0 :
                        for param in model.parameters():
                            L1_loss += torch.sum(torch.abs(param))
                        loss += Lambda_L1*L1_loss
                    loss = loss.mean()
                    loss.backward()
                    optimizer.step()
                    train_loss += loss
#                     loss = None
#                     train_loss += loss.item()

                for idx in range(nb_batch_valid):
                    model.eval()
                    data = X_valid[idx*batch_size:(idx+1)*batch_size]
                    target = Y_valid[idx*batch_size:(idx+1)*batch_size]
                    tData = torch.DoubleTensor(data)
                    tTarget = torch.DoubleTensor(target)
                    tData, tTarget = tData.cuda(), tTarget.cuda() 
                    with torch.no_grad():
                        outputs = model(tData)
                        loss = criterion(outputs, tTarget.reshape(-1,1))
                        if Lambda_AUC > 0 :
                            roc_train = roc_auc_score(tTarget.reshape(-1,1).clone().cpu().data, outputs.clone().cpu().data)
                            AUC_loss = Lambda_AUC*((1-roc_train)/roc_train)
                            loss += loss.clone()*AUC_loss
                        L1_loss = 0
                        if Lambda_L1 > 0 :
                            for param in model.parameters():
                                L1_loss += torch.sum(torch.abs(param))
                            loss += Lambda_L1*L1_loss
#                     valid_loss += loss.item()
                    valid_loss += loss.mean()
                Y_hat = predict([model], X_valid)
                roc_valid = roc_auc_score(Y_valid, Y_hat)

                model.cpu()
                if valid_loss < valid_loss_local_value[model_idx]:
                    valid_loss_local_value[model_idx] = valid_loss
                    if valid_loss < valid_loss_global_value[model_idx]:
                        valid_loss_global_value[model_idx] = valid_loss
#                 else:
#                     countdown[model_idx] -= 1

                saved = " "
                valid_metric = valid_loss
                if P["valid_Metric"] == "auc": valid_metric = (1-roc_valid)
                if valid_metric < valid_metric_local_value[model_idx]:
                    valid_metric_local_value[model_idx] = valid_metric
                    countdown[model_idx] = countdown_init
                    if valid_metric < valid_metric_global_value[model_idx]:
                        found_a_better_model = True
                        recaculate_roc = True                        
                        valid_metric_global_value[model_idx] = valid_metric
                        state_dict = model.state_dict()
                        models_kept[model_idx].load_state_dict(state_dict)
                        saved = "*"
                else:
                    countdown[model_idx] -= 1

                if valid_loss < valid_loss_global:
                    valid_loss_global = valid_loss 

                if roc_valid > roc_valid_value:
                    roc_valid_value = roc_valid 

                if roc >= roc_value :
                    roc_value = roc
                
                found_a_better_model_tag = " "
                if found_a_better_model == True:
                    found_a_better_model_tag = "!"

                print('{}M:{}{} Epoch:{:3}  Tr: {:.4f}  Va: {:.5f} {:.5f} {:.5f}  auc {:.4f} {:.4f}  {}  {}'.format(
                    found_a_better_model_tag, model_idx, saved, epoch, train_loss/nb_batch_train, 
                    valid_loss/nb_batch_valid, valid_loss_local_value[model_idx]/nb_batch_valid, 
                    valid_loss_global_value[model_idx]/nb_batch_valid, 
                    roc_valid, roc, countdown, len(X_train) ))

        if recaculate_roc == True:
            recaculate_roc = False
            Y_hat = predict(models_kept, X)
            roc = roc_auc_score(Y, Y_hat)

        if sum(countdown) == 0:
            break
    
    if found_a_better_model == False:
        break
    
    if P_Boosting :
        print("boosting...")
        for model_idx, model in enumerate(models_kept):
            X_train, Y_train = [], []
            X_valid, Y_valid = [], []
            valid_selected = []
            for j in range(P["data_NbValidFolds"]):
                valid_selected.append((model_idx+j)%len(models))
                X_valid = XL[(model_idx+j)%len(models)]
                Y_valid = YL[(model_idx+j)%len(models)]
                if P_Boosting_Top_valid > 0 :
                    Y_hat = predict([model], X_valid)
                    diff = np.abs(np.array(Y_valid) - Y_hat.reshape(-1))
                    indice_to_boost[(model_idx+j)%len(models)] += [i for i in (diff.argsort()[-P_Boosting_Top_valid:])]
            train_selected = [j for j in range(len(models)) if j not in(valid_selected)]
            for j in train_selected:
                X_train = XL[(j)%len(models)]
                Y_train = YL[(j)%len(models)]
                if P_Boosting_Top_train > 0 :
                    Y_hat = predict([model], X_train)
                    diff = np.abs(np.array(Y_train) - Y_hat.reshape(-1))
                    indice_to_boost[(j)%len(models)] += [i for i in (diff.argsort()[-P_Boosting_Top_train:])]

print("done.... ({:.8f})".format(roc))

------------1
!M:0* Epoch:  1  Tr: 0.6372  Va: 0.49235 0.49235 0.49235  auc 0.8198 0.0000  [5,5,5,5,5,5,5,5,5,5]
!M:1* Epoch:  1  Tr: 0.6286  Va: 0.48641 0.48641 0.48641  auc 0.8246 0.0000  [5,5,5,5,5,5,5,5,5,5]
!M:2* Epoch:  1  Tr: 0.6246  Va: 0.48947 0.48947 0.48947  auc 0.8140 0.0000  [5,5,5,5,5,5,5,5,5,5]
!M:3* Epoch:  1  Tr: 0.6308  Va: 0.49733 0.49733 0.49733  auc 0.8177 0.0000  [5,5,5,5,5,5,5,5,5,5]
!M:4* Epoch:  1  Tr: 0.6267  Va: 0.49248 0.49248 0.49248  auc 0.8264 0.0000  [5,5,5,5,5,5,5,5,5,5]
!M:5* Epoch:  1  Tr: 0.6295  Va: 0.47052 0.47052 0.47052  auc 0.8158 0.0000  [5,5,5,5,5,5,5,5,5,5]
!M:6* Epoch:  1  Tr: 0.6179  Va: 0.46408 0.46408 0.46408  auc 0.8303 0.0000  [5,5,5,5,5,5,5,5,5,5]
!M:7* Epoch:  1  Tr: 0.6298  Va: 0.50120 0.50120 0.50120  auc 0.8235 0.0000  [5,5,5,5,5,5,5,5,5,5]
!M:8* Epoch:  1  Tr: 0.6291  Va: 0.44711 0.44711 0.44711  auc 0.8211 0.0000  [5,5,5,5,5,5,5,5,5,5]
!M:9* Epoch:  1  Tr: 0.6255  Va: 0.45239 0.45239 0.45239  auc 0.8206 0.0000  [5,5,5,5,5,5,5,5,5

!M:2* Epoch:  8  Tr: 0.4618  Va: 0.40147 0.40147 0.40147  auc 0.8607 0.8827  [5,5,5,5,5,5,5,5,5,5]
!M:3  Epoch:  8  Tr: 0.4581  Va: 0.41134 0.40239 0.40239  auc 0.8605 0.8827  [5,5,5,4,5,5,5,5,5,5]
!M:4* Epoch:  8  Tr: 0.4635  Va: 0.42215 0.41343 0.41343  auc 0.8661 0.8827  [5,5,5,4,5,5,5,5,5,5]
!M:5* Epoch:  8  Tr: 0.4624  Va: 0.41298 0.39176 0.39176  auc 0.8641 0.8827  [5,5,5,4,5,5,5,5,5,5]
!M:6* Epoch:  8  Tr: 0.4604  Va: 0.41211 0.39925 0.39925  auc 0.8626 0.8827  [5,5,5,4,5,5,5,5,5,5]
!M:7* Epoch:  8  Tr: 0.4639  Va: 0.38784 0.38784 0.38784  auc 0.8673 0.8827  [5,5,5,4,5,5,5,5,5,5]
!M:8* Epoch:  8  Tr: 0.4623  Va: 0.38111 0.38111 0.38111  auc 0.8665 0.8827  [5,5,5,4,5,5,5,5,5,5]
!M:9* Epoch:  8  Tr: 0.4608  Va: 0.39777 0.39777 0.39777  auc 0.8610 0.8827  [5,5,5,4,5,5,5,5,5,5]
!M:0* Epoch:  9  Tr: 0.4610  Va: 0.41784 0.39221 0.39221  auc 0.8637 0.8838  [5,5,5,4,5,5,5,5,5,5]
!M:1  Epoch:  9  Tr: 0.4628  Va: 0.41228 0.41228 0.41228  auc 0.8667 0.8838  [5,4,5,4,5,5,5,5,5,5]
!M:2* Epoc

!M:4  Epoch: 15  Tr: 0.4455  Va: 0.40124 0.39003 0.39003  auc 0.8677 0.8912  [5,5,5,5,4,5,4,5,5,5]
!M:5* Epoch: 15  Tr: 0.4447  Va: 0.38385 0.38267 0.38267  auc 0.8672 0.8912  [5,5,5,5,4,5,4,5,5,5]
!M:6* Epoch: 15  Tr: 0.4464  Va: 0.38717 0.37800 0.37800  auc 0.8652 0.8912  [5,5,5,5,4,5,5,5,5,5]
!M:7* Epoch: 15  Tr: 0.4396  Va: 0.37787 0.37636 0.37636  auc 0.8695 0.8912  [5,5,5,5,4,5,5,5,5,5]
!M:8* Epoch: 15  Tr: 0.4455  Va: 0.38335 0.37407 0.37407  auc 0.8691 0.8912  [5,5,5,5,4,5,5,5,5,5]
!M:9  Epoch: 15  Tr: 0.4425  Va: 0.37092 0.36345 0.36345  auc 0.8634 0.8912  [5,5,5,5,4,5,5,5,5,4]
!M:0* Epoch: 16  Tr: 0.4417  Va: 0.39315 0.39116 0.39116  auc 0.8660 0.8924  [5,5,5,5,4,5,5,5,5,4]
!M:1* Epoch: 16  Tr: 0.4459  Va: 0.38391 0.38091 0.38091  auc 0.8689 0.8924  [5,5,5,5,4,5,5,5,5,4]
!M:2* Epoch: 16  Tr: 0.4417  Va: 0.38824 0.38747 0.38747  auc 0.8640 0.8924  [5,5,5,5,4,5,5,5,5,4]
!M:3* Epoch: 16  Tr: 0.4427  Va: 0.40764 0.38391 0.38391  auc 0.8639 0.8924  [5,5,5,5,4,5,5,5,5,4]
!M:4  Epoc

!M:6  Epoch: 22  Tr: 0.4327  Va: 0.35448 0.35448 0.35448  auc 0.8657 0.9001  [5,4,5,5,3,5,4,5,3,5]
!M:7* Epoch: 22  Tr: 0.4296  Va: 0.37282 0.35725 0.35725  auc 0.8703 0.9001  [5,4,5,5,3,5,4,5,3,5]
!M:8* Epoch: 22  Tr: 0.4299  Va: 0.35545 0.35545 0.35545  auc 0.8694 0.9001  [5,4,5,5,3,5,4,5,5,5]
!M:9  Epoch: 22  Tr: 0.4284  Va: 0.35908 0.35588 0.35588  auc 0.8644 0.9001  [5,4,5,5,3,5,4,5,5,4]
!M:0  Epoch: 23  Tr: 0.4287  Va: 0.37919 0.36381 0.36381  auc 0.8662 0.9013  [4,4,5,5,3,5,4,5,5,4]
!M:1* Epoch: 23  Tr: 0.4308  Va: 0.38027 0.37373 0.37373  auc 0.8696 0.9013  [4,5,5,5,3,5,4,5,5,4]
!M:2  Epoch: 23  Tr: 0.4285  Va: 0.36496 0.36496 0.36496  auc 0.8650 0.9013  [4,5,4,5,3,5,4,5,5,4]
!M:3  Epoch: 23  Tr: 0.4282  Va: 0.37353 0.35522 0.35522  auc 0.8651 0.9013  [4,5,4,4,3,5,4,5,5,4]
!M:4  Epoch: 23  Tr: 0.4310  Va: 0.37605 0.37320 0.37320  auc 0.8689 0.9013  [4,5,4,4,2,5,4,5,5,4]
!M:5* Epoch: 23  Tr: 0.4281  Va: 0.37798 0.36033 0.36033  auc 0.8689 0.9013  [4,5,4,4,2,5,4,5,5,4]
!M:6  Epoc

!M:1  Epoch: 30  Tr: 0.4172  Va: 0.34206 0.34206 0.34206  auc 0.8695 0.9082  [0,3,4,5,2,2,5,0,5,3]
!M:2  Epoch: 30  Tr: 0.4192  Va: 0.35288 0.33898 0.33898  auc 0.8648 0.9082  [0,3,3,5,2,2,5,0,5,3]
!M:3  Epoch: 30  Tr: 0.4172  Va: 0.34430 0.34430 0.34430  auc 0.8648 0.9082  [0,3,3,4,2,2,5,0,5,3]
!M:4  Epoch: 30  Tr: 0.4165  Va: 0.34910 0.34910 0.34910  auc 0.8691 0.9082  [0,3,3,4,1,2,5,0,5,3]
!M:5  Epoch: 30  Tr: 0.4146  Va: 0.34570 0.34375 0.34375  auc 0.8687 0.9082  [0,3,3,4,1,1,5,0,5,3]
!M:6  Epoch: 30  Tr: 0.4194  Va: 0.35165 0.33684 0.33684  auc 0.8663 0.9082  [0,3,3,4,1,1,4,0,5,3]
!M:8  Epoch: 30  Tr: 0.4147  Va: 0.33757 0.33613 0.33613  auc 0.8697 0.9082  [0,3,3,4,1,1,4,0,4,3]
!M:9  Epoch: 30  Tr: 0.4158  Va: 0.34864 0.34114 0.34114  auc 0.8655 0.9082  [0,3,3,4,1,1,4,0,4,2]
!M:1  Epoch: 31  Tr: 0.4179  Va: 0.33400 0.33400 0.33400  auc 0.8693 0.9082  [0,2,3,4,1,1,4,0,4,2]
!M:2  Epoch: 31  Tr: 0.4152  Va: 0.34489 0.33898 0.33898  auc 0.8658 0.9082  [0,2,2,4,1,1,4,0,4,2]
!M:3  Epoc

In [16]:
Y_hat = predict(models_kept, X)

auc = roc_auc_score(Y, Y_hat)

df_grid = pd.read_csv("grid.csv", index_col=0)
df_grid.loc[grid_index, "auc"] = auc
df_grid.to_csv("grid.csv", index=True)

print(auc)

0.9090868046590606


In [17]:
Y_hat = predict(models_kept, Z)

df_result = df_test[["ID_code"]].copy()
df_result["target"] = pd.Series(Y_hat.reshape(-1))
# df_result.head()
filename = "Prediction_{:03}.csv".format(grid_index)
df_result.to_csv(filename, index=False)
print(filename)

for model_idx, model in enumerate(models_kept):
    Y_hat_m = predict([model], Z)
    df_result = df_test[["ID_code"]].copy()
    df_result["target"] = pd.Series(Y_hat_m.reshape(-1))
    filename = "Prediction_{:03}_M{:02}.csv".format(grid_index, model_idx)
    df_result.to_csv(filename, index=False)
    print(filename)    

Prediction_114.csv
Prediction_114_M00.csv
Prediction_114_M01.csv
Prediction_114_M02.csv
Prediction_114_M03.csv
Prediction_114_M04.csv
Prediction_114_M05.csv
Prediction_114_M06.csv
Prediction_114_M07.csv
Prediction_114_M08.csv
Prediction_114_M09.csv


In [62]:
!kaggle competitions submit -c santander-customer-transaction-prediction -f Prediction_114_XGB.csv -m "XGB only"    

100%|███████████████████████████████████████| 6.08M/6.08M [01:00<00:00, 106kB/s]
Successfully submitted to Santander Customer Transaction Prediction

In [63]:
!kaggle competitions submissions -c santander-customer-transaction-prediction -q

fileName                 date                 description                        status    publicScore  privateScore  
-----------------------  -------------------  ---------------------------------  --------  -----------  ------------  
Prediction_114_XGB.csv   2019-04-06 08:06:53  XGB only                           complete  0.883        None          
Prediction_114_Vote.csv  2019-04-06 08:05:16  vote 4 methods                     complete  0.894        None          
Prediction_114.csv       2019-04-06 07:49:47  real downsampling                  complete  0.876        None          
Prediction_113.csv       2019-03-17 00:54:28  all-in in-loop aug                 complete  0.890        None          
Prediction_112.csv       2019-03-17 00:53:33  quick in-loop aug                  complete  0.888        None          
Prediction_110.csv       2019-03-17 00:52:37  all-in +lgb +xgb over-overfitting  complete  0.872        None          
Prediction_104.csv       2019-03-16 08:46:08  al